In [25]:
from pymatgen.io.vasp import Potcar
import pymatgen

In [26]:
def print_file(fname, nlines=10, header=False):
    with open(fname) as f:
        lines = f.readlines()
        if nlines == 0:
            nlines = len(lines)
        content = "".join(lines[:nlines])
        if len(lines) > nlines:
            content += "..."
        if header:
            print(f"---{fname}---")
        print(content)

In [30]:
from pymatgen.core import Structure, Lattice

lattice = Lattice.cubic(5.64)
structure = Structure(lattice, ["Na", "Cl"], [[0,0,0], [0.5, 0.5, 0.5]])
structure.sort()
print(structure)

Full Formula (Na1 Cl1)
Reduced Formula: NaCl
abc   :   5.640000   5.640000   5.640000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (2)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Na    0    0    0
  1  Cl    0.5  0.5  0.5


In [29]:
# # structure relaxation
# from pymatgen.io.vasp.sets import MPRelaxSet

# vasp_input_set = MPRelaxSet(structure, user_potcar_functional='PBE_54')
# vasp_input_set.write_input(".")

/home2/harsha.vasamsetti/miniconda3/envs/slices/lib/python3.9/site-packages/pymatgen/io/vasp/sets.py:366: BadInputSetWarning: Overriding the POTCAR functional is generally not recommended  as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.
  warnings.warn(


In [18]:
# from pymatgen.io.vasp.sets import MPScanRelaxSet

# vasp_input_set = MPScanRelaxSet(structure, vdw="rVV10")
# vasp_input_set.write_input(".")

In [31]:
from pymatgen.io.vasp.sets import MPStaticSet, MPRelaxSet

user_incar_settings = { 'EDIFF': 1e-5,
                        'EDIFFG': -0.02,
                        'LREAL': False,
                        'NSW': 99,
                        'ALGO': 'Normal',
                        'PREC': 'Accurate',
                        'ISMEAR': 0,
                        'SIGMA': 0.05,
                        'ISIF': 3,
                        'LORBIT': False,
                        'LCHARG': False,
                        'LWAVE': False,
                        'LDAU': False,
                        'ISPIN': 1,
                        'ISYM': 0
                            }

user_kpoint_settings = {'reciprocal_density': 64}

vasp_input_set = MPStaticSet(structure, 
                             user_incar_settings=user_incar_settings,
                             user_kpoints_settings = user_kpoint_settings,
                             user_potcar_functional='PBE_54')

vasp_input_set.write_input(".")

print_file("INCAR", nlines=0, header=True)
print_file("KPOINTS", nlines=10, header=True)

---INCAR---
ALGO = Normal
EDIFF = 1e-05
EDIFFG = -0.02
ENCUT = 520
IBRION = -1
ISIF = 3
ISMEAR = 0
ISPIN = 1
ISYM = 0
LAECHG = True
LASPH = True
LCHARG = False
LORBIT = False
LREAL = False
LVHAR = True
LWAVE = False
MAGMOM = 2*0.6
NELM = 100
NSW = 99
PREC = Accurate
SIGMA = 0.05

---KPOINTS---
pymatgen with grid density = 177 / number of atoms
0
Monkhorst
4 4 4



/home2/harsha.vasamsetti/miniconda3/envs/slices/lib/python3.9/site-packages/pymatgen/io/vasp/sets.py:366: BadInputSetWarning: Overriding the POTCAR functional is generally not recommended  as it significantly affect the results of calculations and compatibility with other calculations done with the same input set. Note that some POTCAR symbols specified in the configuration file may not be available in the selected functional.
  warnings.warn(


In [32]:
# Generate relaxation set input files
vasp_input_set = MPRelaxSet(structure, user_potcar_functional='PBE_54')
vasp_input_set.write_input("./relax")

In [ ]:
# Run VASP
print("\nRunning VASP for structure relaxation...")

In [33]:
import subprocess


vasp_cmd = ["mpirun", "-np", "1", "vasp_gpu"]
try:
    subprocess.run(vasp_cmd, cwd="./relax", check=True)
    print("VASP relaxation completed successfully.")
except subprocess.CalledProcessError as e:
    print(f"Error running VASP: {e}")
    exit(1)

--------------------------------------------------------------------------
mpirun was unable to find the specified executable file, and therefore
did not launch the job.  This error was first reported for process
rank 0; it may have occurred for other processes as well.

NOTE: A common cause for this error is misspelling a mpirun command
      line parameter option (remember that mpirun interprets the first
      unrecognized command line token as the executable).

Node:       gnode114
Executable: vasp_gpu
--------------------------------------------------------------------------
Error running VASP: Command '['mpirun', '-np', '1', 'vasp_gpu']' returned non-zero exit status 134.


In [ ]:
import json
from pymatgen.entries.compatibility import MaterialsProjectCompatibility

with open('./chemPotMP.json') as handle:
        chemPot = json.load(handle)
        

compat = MaterialsProjectCompatibility()
compat.process_entries(entries)
entry = entries[0]

enthalpyForm = entry.energy
temp = entry.composition.get_el_amt_dict()
for element, amount in temp.items():
    enthalpyForm -= amount * chemPot[element]
enthalpyForm /= entry.composition.num_atoms

with open("CONTCAR.relax", "r") as fn2:
    poscar = fn2.read().replace('\n', '\\n')

result_file.write(f"{row[0]},{row[1]},{enthalpyForm},{poscar},{row[4]}\n")

In [1]:
import pandas as pd
pd.read_csv("/scratch/harsha.vasamsetti/results.csv")

,Compound,Spacegroup,Band_Gap,Band_Gap_Type,POSCAR
0,NaCl,Pm-3m,3.9759,M-\Gamma,Na1 Cl1 \n 1...
